# Imports

In [1]:
import re
import nltk
import json
import time
import csv
import concurrent.futures
import os

from concurrent.futures import ThreadPoolExecutor
from deep_translator import GoogleTranslator
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from transformers import pipeline

from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from mpl_toolkits.mplot3d import Axes3D
from itertools import islice

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nilson\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nilson\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
ner_model = pipeline('ner', model='d4data/biomedical-ner-all')

All PyTorch model weights were used when initializing TFDistilBertForTokenClassification.

All the weights of TFDistilBertForTokenClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForTokenClassification for predictions without further training.


In [3]:
bulatorio = pd.read_excel("documentos_bulario.xlsx")
bulatorio.head()

,Unnamed: 0,composicao,nome_medicamento,sentencas_tokenizadas,texto
0,0,composição reumon gel por cada 1 g de gel con...,Reumon Gel,"[['reumon', 'gel', 'indicação', 'para', 'que',...",reumon gel indicaç...
1,1,composição cada comprimido contém: fenitoína ...,Hidantal,"[['hidantal', 'indicação', 'para', 'que', 'ser...",hidantal indicação...
2,2,composição cada comprimido contém 50 mg de di...,Inflamax,"[['inflamax', 'indicação', 'para', 'que', 'ser...",inflamax indicação...
3,3,composição adalat cápsulas cada cápsula de ad...,Adalat,"[['adalat', 'indicação', 'para', 'que', 'serve...",adalat indicação p...
4,4,composição gardasil é composto por proteínas ...,Gardasil,"[['gardasil', 'indicação', 'para', 'que', 'ser...",gardasil indicação...


# Pré processamento

In [4]:
def findText(text, frase1, frase2):
    default = f'{frase1}(.*?){frase2}'
    result = re.search(default, text, re.DOTALL)
    if result:
        return result.group(1)[2:]
    return ''

begin = 'quais os males que pode me causar?'
end = '.   '

dataset = {}

for _, row in bulatorio.iterrows():
    dataset[row['nome_medicamento']] = findText(row['texto'], begin, end)

begin = ' '
end = 'composição'

for key in dataset:
    result = findText(dataset[key], begin, end)

    if(result != ''):
        dataset[key] = result

# Quebrando em sentenças

In [5]:
tokenized_sent = {}
for name, text in dataset.items():
    words = word_tokenize(text)
    sentences = sent_tokenize(text)
    tokenized_sent[name] = sentences

In [6]:
tokenized_sent["Reumon Gel"]

['alguns dos efeitos colaterais de reumon podem incluir vermelhidão, urticária, bolinhas ou inchaço na pele']

# Levando as sentenças para o inglês

In [16]:
tokenized_sent_english = {}

# Verifica se o arquivo já existe
if os.path.exists('tokenized_sent_english.txt'):
    # Carrega os dados do arquivo
    with open('tokenized_sent_english.txt', 'r', encoding='utf-8') as f:
        # Lê cada linha do arquivo
        for line in f:
            # Divide a linha no primeiro ':'
            key, value = line.split(':', 1)
            
            # Remove espaços em branco extras
            key = key.strip()
            value = value.strip()
            
            # Converte a string de lista em uma lista real
            # Isso assume que a lista está formatada como uma lista Python
            value = eval(value)
            
            # Adiciona ao dicionário
            tokenized_sent_english[key] = value
        
        print(len(tokenized_sent_english))
        
else:
    # Função para traduzir uma descrição
    def translate_symptom(text):
        try:
            return GoogleTranslator(source='auto', target='english').translate(text)
        except Exception as e:
            print(f"Erro ao traduzir o sintoma '{text}': {e}")
            return None
        
    # Função para salvar o progresso
    def save_progress():
        with open('tokenized_sent_english.txt', 'w', encoding='utf-8') as f:
            for key, values in tokenized_sent_english.items():
                f.write(f"{key}: {values}\n")

    # Início do tempo
    start = time.time()

    # Cria um executor de thread
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Percorre o dicionário
        for i, (key, value) in enumerate(tqdm(tokenized_sent.items())):
            # Inicializa a lista para a chave atual
            tokenized_sent_english[key] = []
            
            for text in value:
                # Verifique se passou 1 segundo desde o último lote de 5 requisições
                if i % 5 == 0 and time.time() - start < 1:
                    time.sleep(1 - (time.time() - start))
                    start = time.time()

                # Traduza o sintoma
                future = executor.submit(translate_symptom, text)
                try:
                    result = future.result()
                    if result is not None:
                        # Adiciona o resultado à lista para a chave atual
                        tokenized_sent_english[key].append(result)
                except Exception as e:
                    print(f"Erro ao traduzir a descrição do medicamento: '{key}': {e}")
                    save_progress()

    save_progress()
    print(len(tokenized_sent_english))


1562


# Inferindo quais são os efeitos colaterais

In [15]:
# Dicionário para armazenar os sintomas
sintomas = {}

# Verifica se o arquivo já existe
if os.path.exists('sintomas.txt'):
    # Carrega os dados do arquivo
    with open('sintomas.txt', 'r', encoding='utf-8') as f:
        # Lê cada linha do arquivo
        for line in f:
            # Divide a linha no primeiro ':'
            key, value = line.split(':', 1)
            
            # Remove espaços em branco extras
            key = key.strip()
            value = value.strip()
            
            # Converte a string de lista em uma lista real
            # Isso assume que a lista está formatada como uma lista Python
            value = eval(value)
            
            # Adiciona ao dicionário
            sintomas[key] = value

        print(len(sintomas))
        
else:
    # Percorre o dicionário de descrições
    for medicamento, lista_descricoes in tqdm(tokenized_sent_english.items()):
        # Inicializa a lista de sintomas para o medicamento atual
        sintomas[medicamento] = []

        # Percorre a lista de descrições
        for descricao in lista_descricoes:
            # Usa o modelo para fazer a predição
            predicao = ner_model(descricao)

            # Extrai os sintomas da predição com confiança maior ou igual a 60%
            sintomas_descricao = [p['word'] for p in predicao if p['entity'] == 'B-Sign_symptom' and p['score'] >= 0.9]

            # Adiciona os sintomas à lista de sintomas do medicamento
            sintomas[medicamento].extend(sintomas_descricao)

    with open('sintomas.txt', 'w', encoding='utf-8') as f:
            for key, values in sintomas.items():
                f.write(f"{key}: {values}\n")


1562


# Voltando os efeitos colaterais para o português

In [17]:
# Dicionário para armazenar os sintomas
sintomas_pt = {}

# Verifica se o arquivo já existe
if os.path.exists('sintomas_pt.txt'):
    # Carrega os dados do arquivo
    with open('sintomas_pt.txt', 'r', encoding='utf-8') as f:
        # Lê cada linha do arquivo
        for line in f:
            # Divide a linha no primeiro ':'
            key, value = line.split(':', 1)
            
            # Remove espaços em branco extras
            key = key.strip()
            value = value.strip()
            
            # Converte a string de lista em uma lista real
            # Isso assume que a lista está formatada como uma lista Python
            value = eval(value)
            
            # Adiciona ao dicionário
            sintomas_pt[key] = value

        print(len(sintomas_pt))
        
else:
    # Função para traduzir uma descrição
    def translate_symptom(text):
        try:
            return GoogleTranslator(source='auto', target='portuguese').translate(text)
        except Exception as e:
            print(f"Erro ao traduzir o sintoma '{text}': {e}")
            return None
        
    # Função para salvar o progresso
    def save_progress():
        with open('sintomas_pt.txt', 'w', encoding='utf-8') as f:
            for key, values in sintomas_pt.items():
                f.write(f"{key}: {values}\n")

    # Início do tempo
    start = time.time()

    # Cria um executor de thread
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Percorre o dicionário
        for i, (key, value) in enumerate(tqdm(sintomas.items())):
            # Inicializa a lista para a chave atual
            sintomas_pt[key] = []
            
            for text in value:
                # Verifique se passou 1 segundo desde o último lote de 5 requisições
                if i % 5 == 0 and time.time() - start < 1:
                    time.sleep(1 - (time.time() - start))
                    start = time.time()

                # Traduza o sintoma
                future = executor.submit(translate_symptom, text)
                try:
                    result = future.result()
                    if result is not None:
                        # Adiciona o resultado à lista para a chave atual
                        sintomas_pt[key].append(result)
                except Exception as e:
                    print(f"Erro ao traduzir a descrição do medicamento: '{key}': {e}")
                    save_progress()

    save_progress()
    print(len(sintomas_pt))


1562


In [63]:
sintomas_pt["Reumon Gel"]

['vermelho', 'colmeia', 'solavancos', 'inchaço']

# Costrução de um grafo

In [36]:
import pickle
import os
import networkx as nx

# Caminho do arquivo do grafo
arquivo_grafo = 'grafo.pkl'

# Verifica se o arquivo do grafo já existe
if os.path.exists(arquivo_grafo):
    # Carrega o grafo do arquivo
    with open(arquivo_grafo, 'rb') as f:
        G = pickle.load(f)
else:
    # Cria um novo grafo
    G = nx.Graph()

    # Adiciona os nós e as arestas ao grafo
    for medicamento, lista_sintomas in sintomas_pt.items():
        for sintoma in lista_sintomas:
            G.add_edge(medicamento, sintoma)

    # Salva o grafo no arquivo
    with open(arquivo_grafo, 'wb') as f:
        pickle.dump(G, f)


# Amostra do gravo

In [37]:
import plotly.graph_objects as go

# Seleciona uma parte do grafo (por exemplo, os primeiros 10 nós)
nodes = list(G.nodes())[:5]
subgraph = G.subgraph(nodes)

# Cria listas para armazenar os nós e as arestas
nodes = list(subgraph.nodes())
edges = list(subgraph.edges())
colors = ['red' if node in sintomas_pt else 'black' for node in nodes]

# Cria listas para armazenar as coordenadas x, y e z dos nós
x = []
y = []
z = []

# Gera coordenadas aleatórias para cada nó
for _ in nodes:
    x.append(np.random.uniform())
    y.append(np.random.uniform())
    z.append(np.random.uniform())

# Cria um objeto Scatter3d para os nós
nodes_trace = go.Scatter3d(x=x, y=y, z=z, mode='markers+text', text=nodes, marker=dict(size=10, color=colors))

# Cria uma lista para armazenar os objetos Scatter3d para as arestas
edges_traces = []

# Percorre as arestas
for edge in edges:
    # Obtém as coordenadas dos nós da aresta
    x_edge = [x[nodes.index(edge[0])], x[nodes.index(edge[1])]]
    y_edge = [y[nodes.index(edge[0])], y[nodes.index(edge[1])]]
    z_edge = [z[nodes.index(edge[0])], z[nodes.index(edge[1])]]

    # Cria um objeto Scatter3d para a aresta
    edges_trace = go.Scatter3d(x=x_edge, y=y_edge, z=z_edge, mode='lines', line=dict(color='black'))
    edges_traces.append(edges_trace)

# Cria um objeto Figure e adiciona os nós e as arestas
fig = go.Figure(data=[nodes_trace] + edges_traces)

# Mostra o gráfico
fig.show()


In [ ]:
from flask import Flask, request, render_template 
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

@app.route('/') # Use aspas simples ou duplas aqui
def index(): 
  return render_template('index.html')

@app.route('/bfs/', methods=['POST']) # Defina a rota /bfs/ com o método POST
def bfs(): 
  input_string = request.form['input_string'] # Obtenha a string enviada pelo formulário

  symptons = input_string.split(",")
  for i in range(len(symptons)): # Um loop for para cada índice do vetor
    symptons[i] = symptons[i].strip()

  medicines = {}
  for sympton in symptons:
    edges = nx.bfs_edges(G, source=sympton, depth_limit=1)

    for _, medicine in edges:
      medicines[medicine] = medicines.get(medicine, 0) + 1

      medicines_sorted = sorted(medicines.items(), key=lambda x: x[1], reverse=True)
      
  return medicines_sorted # Retorne uma resposta para a rota /bfs/

if __name__ == '__main__':
    app.run(host='0.0.0.0', debug=True, use_reloader=False)